<a href="https://colab.research.google.com/github/brunalves00/brunalves00/blob/main/Extracao_ML_Corrigido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Instalação das Bibliotecas Necessárias

Primeiro, vamos instalar as bibliotecas que precisamos para extração, análise e visualização.

In [21]:
# Instala dependências do sistema
!apt update -y
!apt install -y wget unzip curl gnupg2

# Instala o Google Chrome (já vem na versão 136.x)
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt -fy install
!rm google-chrome-stable_current_amd64.deb

# Extrai a versão instalada do Chrome
!google-chrome --version

# Extrai a versão principal (major) para baixar ChromeDriver correspondente
!CHROME_VERSION=$(google-chrome --version | grep -oP '\d+\.\d+\.\d+') && \
  CHROME_MAJOR=$(echo $CHROME_VERSION | cut -d '.' -f 1) && \
  DRIVER_VERSION=$(curl -s https://chromedriver.storage.googleapis.com/LATEST_RELEASE_$CHROME_MAJOR) && \
  wget -q "https://chromedriver.storage.googleapis.com/$DRIVER_VERSION/chromedriver_linux64.zip" && \
  unzip -q chromedriver_linux64.zip && \
  mv chromedriver /usr/local/bin/chromedriver && \
  chmod +x /usr/local/bin/chromedriver && \
  rm chromedriver_linux64.zip

# Confirma driver e versão
!chromedriver --version


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
89 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources

In [23]:
!pip install requests beautifulsoup4 pandas matplotlib seaborn plotly lxml fake-useragent selenium webdriver-manager cloudscraper

In [24]:
# Instala o Chrome normalmente
!apt update -y
!apt install -y wget unzip curl
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install
!rm google-chrome-stable_current_amd64.deb

# Força instalação do ChromeDriver de versão anterior estável
!wget -q https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!unzip -q chromedriver_linux64.zip
!mv chromedriver /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver
!rm chromedriver_linux64.zip



Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
89 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources

## 2. Configuração dos Headers e User Agents

Definindo uma ampla variedade de user agents e headers para simular diferentes navegadores e dispositivos.

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import random, time

def extract_with_selenium(url):
    try:
        print(f"Tentando extrair com Selenium: {url}")

        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument(f"--user-agent={random.choice(USER_AGENTS)}")
        chrome_options.binary_location = "/usr/bin/google-chrome"

        service = Service("/usr/local/bin/chromedriver")
        driver = webdriver.Chrome(service=service, options=chrome_options)

        driver.get(url)
        time.sleep(5)
        html = driver.page_source
        driver.quit()
        return html
    except Exception as e:
        print(f"Erro no Selenium: {e}")
        return None


In [17]:
def get_page_content(url):
    """Usa apenas Selenium (necessário no Google Colab)"""
    content = extract_with_selenium(url)
    if content:
        return content, "selenium"
    return None, None


In [18]:
from bs4 import BeautifulSoup

def extract_from_html(soup):
    """Extrai dados dos produtos do Mercado Livre com layout poly-component__"""
    products = []
    items = soup.select('div.poly-card__content')

    print(f"Encontrados {len(items)} produtos com .poly-card__content")

    for item in items:
        try:
            title_elem = item.select_one('a.poly-component__title')
            title = title_elem.get_text(strip=True) if title_elem else None
            link = title_elem['href'] if title_elem else None

            seller_elem = item.select_one('span.poly-component__seller')
            seller = seller_elem.get_text(strip=True) if seller_elem else None

            price_elem = item.select_one('div.poly-price__current span[aria-label]')
            price = price_elem['aria-label'] if price_elem else None

            old_price_elem = item.select_one('div.poly-component__price s[aria-label]')
            old_price = old_price_elem['aria-label'] if old_price_elem else None

            products.append({
                'titulo': title,
                'preco': price,
                'preco_antigo': old_price,
                'link': link,
                'vendedor': seller
            })
        except Exception as e:
            print(f"Erro ao processar item: {e}")

    return products


In [19]:
def extract_mercadolivre_data(search_term, max_pages=2):
    from bs4 import BeautifulSoup
    import pandas as pd

    all_products = []

    for page in range(1, max_pages + 1):
        url = f"https://lista.mercadolivre.com.br/{search_term}"
        if page > 1:
            url += f"_Desde_{(page-1)*50+1}"

        print(f"\nExtraindo página {page}: {url}")
        html_content, method = get_page_content(url)

        if not html_content:
            print(f"Não foi possível obter a página {page}")
            continue

        soup = BeautifulSoup(html_content, 'lxml')
        products = extract_from_html(soup)
        print(f"Total extraído na página {page}: {len(products)}")

        all_products.extend(products)

        if page < max_pages:
            delay = random.uniform(3, 7)
            print(f"Aguardando {delay:.2f}s...")
            time.sleep(delay)

    df = pd.DataFrame(all_products)
    print(f"\nTotal final de produtos: {len(df)}")
    return df


In [20]:
search_term = "elsys"  # ou outro termo como "tv", "roteador", etc.
df = extract_mercadolivre_data(search_term, max_pages=2)
df.head()



Extraindo página 1: https://lista.mercadolivre.com.br/elsys
Tentando extrair com Selenium: https://lista.mercadolivre.com.br/elsys
Erro no Selenium: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 136.0.7103.113 with binary path /usr/bin/google-chrome
Stacktrace:
#0 0x5bf8357f64e3 <unknown>
#1 0x5bf835525c76 <unknown>
#2 0x5bf83555304a <unknown>
#3 0x5bf83554e4a1 <unknown>
#4 0x5bf83554b029 <unknown>
#5 0x5bf835589ccc <unknown>
#6 0x5bf83558947f <unknown>
#7 0x5bf835580de3 <unknown>
#8 0x5bf8355562dd <unknown>
#9 0x5bf83555734e <unknown>
#10 0x5bf8357b63e4 <unknown>
#11 0x5bf8357ba3d7 <unknown>
#12 0x5bf8357c4b20 <unknown>
#13 0x5bf8357bb023 <unknown>
#14 0x5bf8357891aa <unknown>
#15 0x5bf8357df6b8 <unknown>
#16 0x5bf8357df847 <unknown>
#17 0x5bf8357ef243 <unknown>
#18 0x7ccbfaff3ac3 <unknown>

Não foi possível obter a página 1

Extraindo página 2: https://lista.mercadolivre.com.br/elsys_Desde_51
Tentando extrair c

""


In [7]:
import random
import requests
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import time
from fake_useragent import UserAgent

# Lista expandida de User Agents modernos e variados
USER_AGENTS = [
    # Chrome em Windows
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    # Firefox em Windows
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0',
    # Edge em Windows
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/123.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/122.0.0.0 Safari/537.36',
    # Safari em macOS
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15',
    # Chrome em macOS
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    # Chrome em Linux
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    # Firefox em Linux
    'Mozilla/5.0 (X11; Linux x86_64; rv:123.0) Gecko/20100101 Firefox/123.0',
    # Chrome em Android
    'Mozilla/5.0 (Linux; Android 13; SM-S908B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 12; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Mobile Safari/537.36',
    # Safari em iOS
    'Mozilla/5.0 (iPhone; CPU iPhone OS 17_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (iPad; CPU OS 17_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1'
]

# Headers completos que simulam um navegador real
DEFAULT_HEADERS = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.mercadolivre.com.br/',
    'Sec-Ch-Ua': '"Chromium";v="123", "Google Chrome";v="123", "Not:A-Brand";v="99"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0',
}

# Função para gerar headers aleatórios
def get_random_headers():
    """Retorna headers com um User-Agent aleatório e outros parâmetros variáveis"""
    user_agent = random.choice(USER_AGENTS)

    # Lista de possíveis valores para Accept-Language
    languages = [
        'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
        'pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3',
        'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7',
        'pt;q=0.9,en-US;q=0.8,en;q=0.7'
    ]

    # Headers base
    headers = {
        'User-Agent': user_agent,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': random.choice(languages),
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://www.mercadolivre.com.br/',
        'Sec-Ch-Ua': '"Chromium";v="123", "Google Chrome";v="123", "Not:A-Brand";v="99"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive'
    }

    # Adicionar ou remover alguns headers aleatoriamente para variar o padrão
    if random.random() > 0.5:
        headers['DNT'] = '1'  # Do Not Track

    if random.random() > 0.7:
        headers['Pragma'] = 'no-cache'

    # Adicionar cookies aleatórios
    if random.random() > 0.6:
        headers['Cookie'] = f'session-id={random.randint(1000000, 9999999)}; csm-hit=tb:{random.randint(1000000, 9999999)}+s-{random.randint(1000000, 9999999)}'

    return headers

## 3. Funções Avançadas para Extração de Dados

Implementando múltiplos métodos de extração para aumentar a robustez contra bloqueios.

In [8]:
# Método 3: Extração com Selenium (para sites com muito JavaScript)
def extract_with_selenium(url):
    """Extrai dados usando Selenium para renderizar JavaScript"""
    try:
        from selenium import webdriver
        from selenium.webdriver.chrome.service import Service
        from webdriver_manager.chrome import ChromeDriverManager
        from selenium.webdriver.chrome.options import Options
        import time

        print(f"Tentando extrair com Selenium: {url}")

        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument(f"--user-agent={random.choice(USER_AGENTS)}")

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        driver.get(url)
        time.sleep(5)  # tempo para renderizar os produtos
        html = driver.page_source
        driver.quit()
        return html
    except Exception as e:
        print(f"Erro no Selenium: {e}")
        return None

# Forçar uso do Selenium, ignorando requests/cloudscraper
def get_page_content(url):
    """Sempre usa Selenium para extrair o conteúdo renderizado dinamicamente"""
    content = extract_with_selenium(url)
    if content:
        return content, "selenium"
    return None, None


## 4. Funções para Extrair Dados do Mercado Livre

Implementando múltiplos métodos de extração de dados dos produtos com seletores CSS atualizados.

In [9]:
def extract_from_html(soup):
    """Extrai dados de produtos com layout poly-component__ usando BeautifulSoup"""
    products = []
    items = soup.select('div.poly-card__content')

    print(f"Encontrados {len(items)} produtos com .poly-card__content")

    for item in items:
        try:
            title_elem = item.select_one('a.poly-component__title')
            title = title_elem.get_text(strip=True) if title_elem else None
            link = title_elem['href'] if title_elem else None

            seller_elem = item.select_one('span.poly-component__seller')
            seller = seller_elem.get_text(strip=True) if seller_elem else None

            price_elem = item.select_one('div.poly-price__current span[aria-label]')
            price = price_elem['aria-label'] if price_elem else None

            old_price_elem = item.select_one('div.poly-component__price s[aria-label]')
            old_price = old_price_elem['aria-label'] if old_price_elem else None

            products.append({
                'titulo': title,
                'preco': price,
                'preco_antigo': old_price,
                'link': link,
                'vendedor': seller
            })
        except Exception as e:
            print(f"Erro ao processar item: {e}")

    return products


## 5. Extrair Dados para um Termo de Busca

Agora vamos extrair dados para um termo de busca específico. Você pode alterar o termo 'elsys' para qualquer outro produto que deseje monitorar.

In [10]:
# Termo de busca
search_term = "elsys"  # Você pode alterar para outro

# Extrair dados com a nova estrutura
df = extract_mercadolivre_data(search_term, max_pages=2)

# Exibir os primeiros resultados
print(df.head())



Extraindo página 1: https://lista.mercadolivre.com.br/elsys
Tentando extrair com Selenium: https://lista.mercadolivre.com.br/elsys
Erro no Selenium: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5875df9a64e3 <unknown>
#1 0x5875df6d5c76 <unknown>
#2 0x5875df6fc757 <unknown>
#3 0x5875df6fb029 <unknown>
#4 0x5875df739ccc <unknown>
#5 0x5875df73947f <unknown>
#6 0x5875df730de3 <unknown>
#7 0x5875df7062dd <unknown>
#8 0x5875df70734e <unknown>
#9 0x5875df9663e4 <unknown>
#10 0x5875df96a3d7 <unknown>
#11 0x5875df974b20 <unknown>
#12 0x5875df96b023 <unknown>
#13 0x5875df9391aa <unknown>
#14 0x5875df98f6b8 <unknown>
#15 0x5875df98f847 <unknown>
#16 0x5875df99f243 <unknown>
#17 0x7f103937eac3 <unknown>

Não foi possível obter o conteúdo da página 1

Extraindo página 2: https://lista.mercadolivre.com.br/elsys_Desde_51
Tentando extrair com Selenium: https://lista.mercadolivre.com.br/elsys_Desde_51
Erro no Selenium: Message: unknown error: cannot find Chrome binary
Stacktrace:

## 6. Salvar os Dados Extraídos em CSV

Vamos salvar os dados extraídos em um arquivo CSV para uso posterior.

In [ ]:
# Verificar se temos dados
if len(df) > 0:
    # Salvar em CSV
    csv_filename = f"{search_term}_produtos.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Dados salvos em {csv_filename}")

    # Fazer download do arquivo
    from google.colab import files
    files.download(csv_filename)
else:
    print("Nenhum produto foi extraído. Não é possível salvar o CSV.")

## 7. Limpeza e Preparação dos Dados

Vamos limpar e preparar os dados para análise.

In [ ]:
# Verificar se temos dados para processar
if len(df) > 0:
    # Verificar valores nulos
    print("Valores nulos por coluna:")
    print(df.isnull().sum())

    # Remover linhas com valores nulos (se necessário)
    df_clean = df.dropna(subset=['titulo', 'preco'])
    print(f"\nTotal de produtos após remoção de nulos: {len(df_clean)}")

    # Converter preço para numérico (removendo possíveis caracteres não numéricos)
    def clean_price(price):
        if pd.isna(price):
            return None
        # Remove caracteres não numéricos, mantendo ponto e vírgula
        price_str = re.sub(r'[^0-9.,]', '', str(price))
        # Substitui vírgula por ponto
        price_str = price_str.replace(',', '.')
        try:
            return float(price_str)
        except:
            return None

    df_clean['preco_num'] = df_clean['preco'].apply(clean_price)
    df_clean.head()
else:
    print("Não há dados para processar. Execute a extração novamente.")

## 8. Análise Exploratória e Visualizações

Vamos criar algumas visualizações para entender melhor os dados.

In [ ]:
# Verificar se temos dados para visualizar
if 'df_clean' in locals() and len(df_clean) > 0:
    # Estatísticas descritivas dos preços
    print("Estatísticas descritivas dos preços:")
    print(df_clean['preco_num'].describe())

    # Importar bibliotecas de visualização
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Configurar matplotlib para exibir corretamente caracteres especiais
    plt.rcParams['font.family'] = 'DejaVu Sans'

    # Histograma de preços
    plt.figure(figsize=(12, 6))
    sns.histplot(df_clean['preco_num'].dropna(), bins=20, kde=True)
    plt.title('Distribuição de Preços dos Produtos')
    plt.xlabel('Preço (R$)')
    plt.ylabel('Frequência')
    plt.grid(True, alpha=0.3)
    plt.show()

    # Boxplot de preços
    plt.figure(figsize=(10, 6))
    sns.boxplot(y=df_clean['preco_num'].dropna())
    plt.title('Boxplot de Preços')
    plt.ylabel('Preço (R$)')
    plt.grid(True, alpha=0.3)
    plt.show()

    # Top 10 vendedores por número de produtos (se houver dados de vendedor)
    if 'vendedor' in df_clean.columns and df_clean['vendedor'].notna().sum() > 0:
        vendedor_counts = df_clean['vendedor'].value_counts().head(10)

        plt.figure(figsize=(12, 6))
        sns.barplot(x=vendedor_counts.values, y=vendedor_counts.index)
        plt.title('Top 10 Vendedores por Número de Produtos')
        plt.xlabel('Número de Produtos')
        plt.ylabel('Vendedor')
        plt.grid(True, alpha=0.3)
        plt.show()
else:
    print("Não há dados limpos para visualizar. Execute a extração e limpeza novamente.")

## 9. Visualizações Interativas com Plotly

Vamos criar algumas visualizações interativas para uma análise mais detalhada.

In [ ]:
# Verificar se temos dados para visualizar
if 'df_clean' in locals() and len(df_clean) > 0:
    import plotly.express as px
    import plotly.graph_objects as go

    # Histograma interativo de preços
    fig = px.histogram(df_clean, x='preco_num', nbins=20, title='Distribuição de Preços (Interativo)')
    fig.update_layout(xaxis_title='Preço (R$)', yaxis_title='Frequência')
    fig.show()

    # Scatter plot de produtos por preço (com hover para ver o título)
    fig = px.scatter(df_clean, x=df_clean.index, y='preco_num', hover_name='titulo',
                     title='Produtos por Preço', labels={'index': 'Índice do Produto', 'preco_num': 'Preço (R$)'})
    fig.update_layout(hovermode='closest')
    fig.show()
else:
    print("Não há dados limpos para visualizar. Execute a extração e limpeza novamente.")

## 10. Análise de Palavras-chave nos Títulos

Vamos analisar as palavras mais comuns nos títulos dos produtos.

In [ ]:
# Verificar se temos dados para analisar
if 'df_clean' in locals() and len(df_clean) > 0 and 'titulo' in df_clean.columns:
    # Análise de palavras-chave nos títulos
    from collections import Counter
    import re

    # Função para extrair palavras dos títulos
    def extract_words(title):
        if pd.isna(title):
            return []
        # Remove caracteres especiais e converte para minúsculas
        words = re.findall(r'\b\w+\b', title.lower())
        # Remove palavras muito comuns (stopwords)
        stopwords = ['de', 'da', 'do', 'e', 'com', 'para', 'em', 'a', 'o', 'as', 'os', 'no', 'na', 'um', 'uma']
        return [word for word in words if word not in stopwords and len(word) > 2]

    # Extrair todas as palavras dos títulos
    all_words = []
    for title in df_clean['titulo'].dropna():
        all_words.extend(extract_words(title))

    # Contar frequência das palavras
    word_counts = Counter(all_words)
    top_words = word_counts.most_common(20)

    # Criar DataFrame para visualização
    words_df = pd.DataFrame(top_words, columns=['Palavra', 'Frequência'])

    # Plotar gráfico de barras
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Frequência', y='Palavra', data=words_df)
    plt.title('Top 20 Palavras mais Frequentes nos Títulos')
    plt.xlabel('Frequência')
    plt.ylabel('Palavra')
    plt.grid(True, alpha=0.3)
    plt.show()

    # Versão interativa com Plotly
    fig = px.bar(words_df, x='Frequência', y='Palavra', title='Top 20 Palavras mais Frequentes nos Títulos (Interativo)')
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    fig.show()
else:
    print("Não há dados de títulos para analisar. Execute a extração e limpeza novamente.")

## 11. Conclusão

Neste notebook, realizamos:
1. Extração de dados do Mercado Livre usando múltiplos métodos (requests, cloudscraper, selenium)
2. Implementação de técnicas avançadas para evitar bloqueios (rotação de user agents, headers variáveis)
3. Extração de dados tanto do HTML quanto do JavaScript embutido
4. Salvamento dos dados em CSV
5. Limpeza e preparação dos dados
6. Análise exploratória com visualizações estáticas e interativas
7. Análise de palavras-chave nos títulos dos produtos

Para um monitoramento contínuo, você pode:
1. Executar este notebook periodicamente
2. Salvar os resultados com timestamp
3. Comparar os preços ao longo do tempo
4. Configurar alertas para mudanças significativas de preço

In [43]:
# Instalar bibliotecas Python
!pip install -q requests beautifulsoup4 pandas matplotlib seaborn plotly lxml fake-useragent selenium webdriver-manager cloudscraper

# Instalar Google Chrome + ChromeDriver correspondente (versão compatível)
!apt update -y
!apt install -y wget unzip curl gnupg2
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt -fy install
!rm google-chrome-stable_current_amd64.deb

# Obter e instalar a versão compatível do ChromeDriver
!CHROME_VERSION=$(google-chrome --version | grep -oP '\d+\.\d+\.\d+') && \
  CHROME_MAJOR=$(echo $CHROME_VERSION | cut -d '.' -f 1) && \
  DRIVER_VERSION=$(curl -s https://chromedriver.storage.googleapis.com/LATEST_RELEASE_$CHROME_MAJOR) && \
  wget -q "https://chromedriver.storage.googleapis.com/$DRIVER_VERSION/chromedriver_linux64.zip" && \
  unzip -q chromedriver_linux64.zip && \
  mv chromedriver /usr/local/bin/chromedriver && \
  chmod +x /usr/local/bin/chromedriver && \
  rm chromedriver_linux64.zip


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,547 kB]
Fetched 1,804 kB in 3s (657 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state informat

In [45]:
!sudo apt remove -y google-chrome-stable


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libvulkan1 mesa-vulkan-drivers
Use 'sudo apt autoremove' to remove them.
The following packages will be REMOVED:
  google-chrome-stable
0 upgraded, 0 newly installed, 1 to remove and 89 not upgraded.
After this operation, 381 MB disk space will be freed.
(Reading database ... 126258 files and directories currently installed.)
Removing google-chrome-stable (136.0.7103.113-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...


In [47]:
!wget -q https://storage.googleapis.com/chrome-for-testing-public/114.0.5735.90/linux64/chrome-linux64.zip
!unzip -q chrome-linux64.zip
!mv chrome-linux64 /opt/chrome
!ln -sf /opt/chrome/chrome /usr/bin/google-chrome
!google-chrome --version
!chromedriver --version


Google Chrome for Testing 114.0.5735.90 
ChromeDriver 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})


In [51]:
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Lista de User-Agents
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 13; SM-S908B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Mobile Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 17_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile Safari/604.1'
]

def extract_with_selenium(url):
    """Extrai dados renderizados via JavaScript usando Selenium"""
    try:
        print(f"Tentando extrair com Selenium: {url}")
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument(f"--user-agent={random.choice(USER_AGENTS)}")
        chrome_options.binary_location = "/usr/bin/google-chrome"
        service = Service("/usr/local/bin/chromedriver")
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.get(url)
        time.sleep(5)
        html = driver.page_source
        driver.quit()
        return html
    except Exception as e:
        print(f"Erro no Selenium: {e}")
        return None

def get_page_content(url):
    """Força Selenium para Colab"""
    content = extract_with_selenium(url)
    if content:
        return content, "selenium"
    return None, None

def extract_from_html(soup):
    """Extrai produtos do Mercado Livre no layout poly-*"""
    products = []
    items = soup.select('div.poly-card__content')
    print(f"Encontrados {len(items)} produtos com .poly-card__content")

    for item in items:
        try:
            title_elem = item.select_one('a.poly-component__title')
            title = title_elem.get_text(strip=True) if title_elem else None
            link = title_elem['href'] if title_elem else None

            seller_elem = item.select_one('span.poly-component__seller')
            seller = seller_elem.get_text(strip=True) if seller_elem else None

            price_elem = item.select_one('div.poly-price__current span[aria-label]')
            price = price_elem['aria-label'] if price_elem else None

            old_price_elem = item.select_one('div.poly-component__price s[aria-label]')
            old_price = old_price_elem['aria-label'] if old_price_elem else None

            products.append({
                'titulo': title,
                'preco': price,
                'preco_antigo': old_price,
                'link': link,
                'vendedor': seller
            })
        except Exception as e:
            print(f"Erro ao processar item: {e}")

    return products

def extract_mercadolivre_data(search_term, max_pages=2):
    """Executa scraping de produtos do Mercado Livre"""
    all_products = []

    for page in range(1, max_pages + 1):
        url = f"https://lista.mercadolivre.com.br/{search_term}"
        if page > 1:
            url += f"_Desde_{(page-1)*50+1}"
        print(f"\nExtraindo página {page}: {url}")
        html_content, method = get_page_content(url)
        if not html_content:
            print(f"Não foi possível obter a página {page}")
            continue

        soup = BeautifulSoup(html_content, 'lxml')
        products = extract_from_html(soup)
        print(f"Total extraído na página {page}: {len(products)}")
        all_products.extend(products)

        if page < max_pages:
            delay = random.uniform(3, 6)
            print(f"Aguardando {delay:.2f}s...")
            time.sleep(delay)

    df = pd.DataFrame(all_products)
    print(f"\nTotal final de produtos extraídos: {len(df)}")
    return df


In [50]:
# Termo de busca
search_term = "elsys"  # ou outro termo

# Executar scraping
df = extract_mercadolivre_data(search_term, max_pages=1000)

# Mostrar resultados
df.head()



Extraindo página 1: https://lista.mercadolivre.com.br/elsys
Tentando extrair com Selenium: https://lista.mercadolivre.com.br/elsys
Encontrados 51 produtos com .poly-card__content
Total extraído na página 1: 51
Aguardando 5.93s...

Extraindo página 2: https://lista.mercadolivre.com.br/elsys_Desde_51
Tentando extrair com Selenium: https://lista.mercadolivre.com.br/elsys_Desde_51
Encontrados 51 produtos com .poly-card__content
Total extraído na página 2: 51
Aguardando 3.03s...

Extraindo página 3: https://lista.mercadolivre.com.br/elsys_Desde_101
Tentando extrair com Selenium: https://lista.mercadolivre.com.br/elsys_Desde_101
Encontrados 51 produtos com .poly-card__content
Total extraído na página 3: 51
Aguardando 4.31s...

Extraindo página 4: https://lista.mercadolivre.com.br/elsys_Desde_151
Tentando extrair com Selenium: https://lista.mercadolivre.com.br/elsys_Desde_151
Encontrados 51 produtos com .poly-card__content
Total extraído na página 4: 51
Aguardando 5.41s...

Extraindo página 

KeyboardInterrupt: 